# **Item 3 - Sobre GenAI e LLMs**

## Problema

<p align="justify"> O Dataset disponibilizado tem 2 colunas de texto, uma de título e outra de descrição. Utilize um LLM - como  ChatGPT ou Claude - para criar features dos produtos que serão úteis para análise. </p>


<a name='1'></a>

## 1 - Carregando dados e modelo

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModel
import torch
import os
import pandas as pd

/home/bruno/BRUNO_DIFRANCO_DDF_DATAANALYST_102023/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
os.chdir("..")

<a name='2'></a>

## 2 - Dataset e LLM loading

Carregando os dados e o modelo.

In [23]:
dataset = load_dataset("spacemanidol/product-search-corpus", 
                       split="train")
dataset

Dataset({
    features: ['docid', 'title', 'text'],
    num_rows: 1118658
})

In [42]:
type(dataset)

datasets.arrow_dataset.Dataset

In [24]:
tokenizer = AutoTokenizer.from_pretrained("spacemanidol/trec-product-search-e5-small-v2")
model = AutoModel.from_pretrained("spacemanidol/trec-product-search-e5-small-v2")

Com um pouco de pesquisa é possível perceber que o dataset foi extraído da Amazon, por isso utilizei as seguintes categorias possíveis de produtos para serem utilizadas no prompt do modelo:

- "Food",
- "Health, Beauty",
- "Electronics, Computers",
- "Home and Kitchen",
- "Clothing",
- "Sports",
- "Office",
- "Movies, Music",
- "Books"     

Decidi utilizar um modelo do mesmo autor que fez o upload dos dados no HuggingFace, e segundo a tag desse [modelo](https://huggingface.co/spacemanidol/trec-product-search-e5-small-v2) ele é feito para "Feature Extraction", que é exatamente o objetivo nesse caso.

Obs: Vou me limitar ao uso da LLM para extrair as categorias de cada produto, e não features específicas para cada produto, pois:
- Há uma dificuldade computacional de executar essa ação em tão pouco tempo, com apenas uma placa de vídeo RTX 3050 4GB, que é o que disponho no momento.
- Há uma limitação de recursos em termos de modelo para executar tal ação, pois as LLM's que conseguiriam fazer esse processo de forma quase perfeita seriam um gpt-3.5-turbo ou gpt-4, ou modelos parecidos, que não são gratuitos. 
  

<a name='2'></a>

## 3 - Utilizando o modelo para fazer inferência

Importante observar que o comando (prompt) que passamos para o modelo é o seguinte:

`"Classify the following product title into its category:{Título do produto}`

`Candidate categories: [Lista de categorias possíveis da Amazon]"`


In [44]:
if torch.cuda.is_available(): # using gpu
    model.to('cuda')            
    predictions = []
    
    categories = {
                0: "Food",
                1: "Health, Beauty",
                2: "Electronics, Computers",
                3: "Home and Kitchen",
                4: "Clothing",
                5: "Sports",
                6: "Office",
                7: "Movies, Music",
                8: "Books"     
            }
    
    for i, example in enumerate(dataset):
        title = example["title"]

        # prompt
        prompt = f"Classify the following product title into its category:\n\n{title}\n\nCandidate categories: {list(categories.values())}."

        # inputs
        inputs = tokenizer(prompt, return_tensors="pt").to('cuda')  # Mova os inputs para a GPU
        
        # pooler_output
        with torch.no_grad():
            model_outputs = model(**inputs)
            pooler_output = model_outputs.pooler_output

        # custom classifier
        class CustomClassifier(torch.nn.Module):
            """
                Personalized classification class. 
                Used for classifying product categories                  
            
            """
            def __init__(self):
                super(CustomClassifier, self).__init__()
                self.num_classes = len(categories)
                self.classifier = torch.nn.Linear(
                                                    model.config.hidden_size, self.num_classes
                                                    ).to('cuda')

            def forward(self, pooled_output):
                return self.classifier(pooled_output)
        
        classifier = CustomClassifier()
        outputs = classifier(pooler_output)

        predicted_class = torch.argmax(outputs, dim=1).item()              
        predictions.append(predicted_class)

        # predicted categories
        predicted_categories = [categories[index] for index in predictions]
        

        print(f"Observation {i + 1}/{len(dataset)} processed")
    


Observation 1/1118658 processed
Observation 2/1118658 processed
Observation 3/1118658 processed
Observation 4/1118658 processed
Observation 5/1118658 processed
Observation 6/1118658 processed
Observation 7/1118658 processed
Observation 8/1118658 processed
Observation 9/1118658 processed
Observation 10/1118658 processed
Observation 11/1118658 processed
Observation 12/1118658 processed
Observation 13/1118658 processed
Observation 14/1118658 processed
Observation 15/1118658 processed
Observation 16/1118658 processed
Observation 17/1118658 processed
Observation 18/1118658 processed
Observation 19/1118658 processed
Observation 20/1118658 processed
Observation 21/1118658 processed
Observation 22/1118658 processed
Observation 23/1118658 processed
Observation 24/1118658 processed
Observation 25/1118658 processed
Observation 26/1118658 processed
Observation 27/1118658 processed
Observation 28/1118658 processed
Observation 29/1118658 processed
Observation 30/1118658 processed
Observation 31/1118

KeyboardInterrupt: 

In [46]:
type(predicted_categories)

list

**No arquivo `llm_pipeline.py` está a pipeline completa e organizada da inferência do modelo de LLM.**

Unindo as previsões com os dados originais:

In [35]:
data = {'category': predicted_categories}
df_category = pd.DataFrame(data)
df_final = pd.concat([dataset, df_category],axis=1)

Salvando em `.parquet`, para economizarmos espaço.

In [100]:
df_final.to_parquet('categories.parquet', index=False, compression="gzip")

## Próximos passos

Com mais tempo seria possível incrementar a solução sem necessariamente ter que gastar com um modelo pago, fazendo, por exemplo, o seguinte:

- Classificando manualmente exemplos para incrementar o prompt
- Utilizar PEFT (Parameter Efficient Fine-Tuning) que busca fazer o fine tuning apenas dos parâmetros "mais relevantes" (entre muitas aspas) do LLM, como 1% dos parâmetros mais relevantes, por exemplo, ao invés de aplicar o fine tuning em todos os parâmetros.